In [1]:
import glob
import cv2 
import os
import keras
import numpy as np
import shutil
import itertools
# import imutils
import matplotlib.pyplot as plt
# from skimage.measure import compare_ssim as ssim
from random import shuffle

from keras import layers
from keras.callbacks import TensorBoard
from keras.layers import Input, Conv2D, MaxPooling2D, MaxPooling1D, UpSampling2D, Conv2DTranspose, Flatten, GRU, Conv1D, UpSampling1D, Reshape, Concatenate, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.backend import squeeze
from ann_visualizer.visualize import ann_viz

Using TensorFlow backend.


In [ ]:
# Cropping out images to get only and clear images of brain 
def crop_images(img):
    img = cv2.resize(img, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray_img, (5, 5), 0)

    thresh_img = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    eroded = cv2.erode(thresh_img, None, iterations=2)
    dilated = cv2.dilate(eroded, None, iterations=2)

    contours = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contour_area = max(contours, key=cv2.contourArea)

    extreme_left = tuple(contour_area[contour_area[:, :, 0].argmin()][0])
    extreme_right = tuple(contour_area[contour_area[:, :, 0].argmax()][0])
    extreme_top = tuple(contour_area[contour_area[:, :, 1].argmin()][0])
    extreme_bottom = tuple(contour_area[contour_area[:, :, 1].argmax()][0])

    contour_img = cv2.drawContours(img.copy(), [contour_area], -1, (0, 255, 255), 4)

    contour_point = cv2.circle(contour_img.copy(), extreme_left, 8, (0, 0, 255), -1)
    contour_point = cv2.circle(contour_point, extreme_right, 8, (0, 255, 0), -1)
    contour_point = cv2.circle(contour_point, extreme_top, 8, (255, 0, 0), -1)
    contour_point = cv2.circle(contour_point, extreme_bottom, 8, (255, 255, 0), -1)

    new_img = img[extreme_top[1]:extreme_bottom[1], extreme_left[0]:extreme_right[0]].copy()
    return new_img

In [ ]:
# Accessing dataset from folder
def load_dataset_train():
  path_train = glob.glob("/content/sample_data/train/*")
  X_train = []
  shuffle(path_train)
  for img in path_train:
    n = cv2.imread(img)
    n_img=crop_images(n)
    X_train.append(n_img)
  return np.array(X_train)

def load_dataset_test():
  path_test = glob.glob("/content/sample_data/test/*")
  X_test = []
  shuffle(path_test)
  for img in path_test:
    n = cv2.imread(img)
    n_img=crop_images(n)
    X_test.append(n_img)
  return np.array(X_test)

In [ ]:
# Preprocessing Images by resizing them and normalizing them 
def preprocess_images(input_images, img_size):
    new_images = []
    for img in input_images:
        img = cv2.resize(img,dsize=img_size,interpolation=cv2.INTER_CUBIC)
        img=img.astype('float32') / 255
        new_images.append(img)
    return np.array(new_images)

In [ ]:
# Loading Datasets and Plotting Images
X_train_load=load_dataset_train()
X_test_load=load_dataset_test()

plt.figure(figsize=(5,5))
plt.axis('off')
plt.imshow(X_train_load[0], cmap='gray')

# Preporocessing and Plotting Images
X_train=preprocess_images(X_train_load,(256,256))
X_test=preprocess_images(X_test_load,(256,256))

plt.figure(figsize=(5,5))
plt.axis('off')
plt.imshow(X_train[0], cmap='gray')
print(X_train[1].shape)

In [ ]:
# Method to add noise to images
def add_noise(input_images):
  new_images=[]
  for img in input_images:
    img1 = img.copy()
    rows,cols = img1.shape[0],img1.shape[1]
    img1= img1+ 0.1 * np.random.normal(loc=0.0, scale=1.0, size=img1.shape)
    new_images.append(img1)
  return new_images

In [ ]:
# Adding Noise 
X_train_noise=add_noise(X_train)
X_test_noise=add_noise(X_test)

# Training Image
plt.figure(figsize=(5,5))
plt.axis('off')
plt.imshow(X_train_noise[0], cmap='gray')

# Testing image
plt.figure(figsize=(5,5))
plt.axis('off')
plt.imshow(X_test_noise[0], cmap='gray')

In [2]:
# Denoising Autoencoder with Recurrent Skip Connections:
# x_train=np.squeeze(X_train_noise)
# y=np.squeeze(X_train)
input_img = Input(shape=(256, 256,3))  

x = Conv2D(32, (3, 3),strides=1, activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3),strides=1, activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), strides=1, activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), strides=1, activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same', name = 'latent_cnn')(x)
print('encoded.shape',encoded.shape)


shaped=Reshape((16*16*64,1))(encoded)
print('shaped.shape:',shaped.shape)

# change value of strides to adjust the context look behind
conv1 = Conv1D(1024,kernel_size=3,strides=1,activation='relu',padding='same')(shaped) 
conv2 = Conv1D(512,kernel_size=3,strides=2,activation='relu',padding='same')(conv1)
conv2_b = MaxPooling1D(2)(conv2)
print('conv2_b.shape:',conv2_b.shape)
conv3 = Conv1D(256,kernel_size=3,strides=2,activation='relu',padding='same')(conv2) 
conv4 = Conv1D(256,kernel_size=1,strides=1,activation='relu',padding='same')(conv3) 
print('conv3.shape:',conv3.shape)

#transpose
Tconv4 = UpSampling1D(size=2)(conv4) 
Tconv4 = Conv1D(512,kernel_size=3,strides=2,activation='relu',padding='same')(Tconv4) 
print('Tconv4.shape:',Tconv4.shape)
#transpose
Tconv5 = UpSampling1D(size=2)(Tconv4) 
Tconv5 = Conv1D(1024,kernel_size=3,strides=2,activation='relu',padding='same')(Tconv5) 
print('Tconv5.shape:',Tconv5.shape)


gru1 = GRU(512,activation='tanh', recurrent_activation='sigmoid',name='latent_rnn',return_sequences=True)(conv3)
# gru1=Reshape((1,512))(gru1)
print('gru1.shape:',gru1.shape)
skip_1 = concatenate([Tconv4, gru1])
print('skip_1.shape:',skip_1.shape) 

gru2 = GRU(1024,activation='tanh', recurrent_activation='sigmoid',name='latent_rnn',return_sequences=True)(conv2_b)
# gru1=Reshape((1,512))(gru1)
print('gru2.shape:',gru2.shape)
skip_2 = concatenate([Tconv5, gru2])
print('skip_2.shape:',skip_2.shape) 


skip_rnn = Conv1D(256,kernel_size=3, strides=1,activation='relu',padding='same')(skip_2)
print('skip_rnn.shape:',skip_rnn.shape) 
# gru2 = GRU(512,activation='tanh', recurrent_activation='sigmoid', return_sequences=True)(shaped)
# print('gru2.shape:',gru2.shape) 
# merge_2 = concatenate([x,gru2])

shaped=Reshape((64,64,256))(skip_rnn)
print('shaped.shape:',shaped.shape)

x = Conv2DTranspose(64, (3, 3),strides=1, activation='relu', padding='same')(shaped)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(64, (3, 3),strides=1, activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(x)
# x = Conv2DTranspose(32, (3, 3),strides=1, activation='relu', padding='same')(x)
# x = UpSampling2D((2, 2))(x)
# x = Conv2DTranspose(32, (3, 3),strides=1, activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2DTranspose(3, (3, 3),strides=1, activation='sigmoid', padding='same')(x)

denoise_recur_auto_encoder = Model(input_img, decoded)
denoise_recur_auto_encoder.summary()
denoise_recur_auto_encoder.compile(optimizer=Adam(), loss='mean_squared_error')



encoded.shape (?, 16, 16, 64)
shaped.shape: (?, 16384, 1)
conv2_b.shape: (?, 4096, 512)
conv3.shape: (?, 4096, 256)
Tconv4.shape: (?, 4096, 512)
Tconv5.shape: (?, 4096, 1024)
gru1.shape: (?, ?, 512)
skip_1.shape: (?, 4096, 1024)
gru2.shape: (?, ?, 1024)
skip_2.shape: (?, 4096, 2048)
skip_rnn.shape: (?, 4096, 256)
shaped.shape: (?, 64, 64, 256)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 32) 0      

In [5]:
ann_viz(denoise_recur_auto_encoder, title="My first neural network")

ImportError: No module named 'graphviz'